In [5]:
import datetime
import json

import requests
from flask import Flask, request
from ImageCollection import ImageCollection
from ImageJson import ImageJSON
from CollectionAlbum import CollectionAlbum
from datetime import datetime

def check_date(date):
    try:
        year, month, day = date.split("-")
        if len(year) != 4 or len(month) != 2 or len(day) != 2:
            return False
        int(year)
        int(month)
        int(day)
        if int(year) < 1995 or (int(year) == 1995 and (int(month) < 6 or (int(month) == 6 and int(day) < 16))):
            return False
        today = datetime.now().date()
        date = datetime.strptime(date, '%Y-%m-%d').date()
        if date > today:
            return False
        return True
    except ValueError:
        return False

def create_example_collections():
    url = 'https://api.nasa.gov/planetary/apod?api_key=bYgUJCu6QjKcvu7WSAElha7ZFNzZq3Dibeq9vYMV&start_date=2023-01-01&end_date=2023-01-05'
    response = requests.get(url)
    data = json.loads(response.text)
    my_collection = ImageCollection("collection 1")
    my_collection2 = ImageCollection("collection 2")
    my_collection3 = ImageCollection("collection 3")
    for i in data:
        apod = ImageJSON(i)
        my_collection.add_obj(apod)
        my_collection2.add_obj(apod)
        my_collection3.add_obj(apod)
    all_collections = CollectionAlbum()
    all_collections.add(my_collection)
    all_collections.add(my_collection2)
    all_collections.add(my_collection3)
    return all_collections

app = Flask(__name__)
all_collections = create_example_collections()


@app.route('/')
def home():
    collection_link = "<br></br>"
    for i in all_collections.image_collections:
        collection_link = collection_link + (
            "<a href='/collection?collection_name={0}'> {0} </a><br></br>".format(i.get_name()))
    create_collection = """
            <form action="/createcollection?" method="POST">
                        New Collection Name:
                        <input type='text' name='name'>
                        Date of first image:
                        <input type='text' name='date'>(YYYY-MM-DD)
                        <input type='submit' value='Create Collection'> (Date must be after 1995-06-16)
                     </form>
        """
    return """
         <html><body>
             <h2><b> Astronomy Picture of the Day</b></h2>
             Existing Collections:
             {1}
             <br>
             Create New Collection
             {0}   
         </body></html>
            """.format(create_collection, collection_link)


@app.route('/collection', methods=["GET"])
def view_collection():
    collection_name = request.args.get('collection_name')
    collection = all_collections.get_collection(collection_name)
    obj_link = "<br></br>"
    for obj in collection.get_obj_list():
        obj_link = obj_link + (
            "<a href='/obj?obj_title={0}&collection_name={1}'> {0} </a><br></br>".format(obj.get_title(),
                                                                                         collection_name))

    add_range_collection = """
                <form action="/add_range_collection?collection_name={0}" method="POST">
                            Begging of range:
                            <input type='text' name='start date'>(YYYY-MM-DD)
                            End of range:
                            <input type='text' name='end date'>(YYYY-MM-DD)
                            <input type='submit' value='Add to Collection'> (Date must be after 1995-06-16)
                         </form>
                         <br>
            """.format(collection_name)
    add_single_collection = """
                    <form action="/add_single_collection?collection_name={0}" method="POST">
                        Single Date:
                        <input type='text' name='date'>(YYYY-MM-DD)
                        <input type='submit' value='Add to Collection'> (Date must be after 1995-06-16)
                     </form>""".format(collection_name)

    sort_list = """
                <form action="/sort?collection_name={0}" method="POST">
                <input type="radio" id="title" name="sort_by" value="title">
                <label for="title">Sort by title</label><br>
                <input type="radio" id="date" name="sort_by" value="date">
                <label for="date">Sort by date</label><br>
                <input type='submit' value='Sort'>
                </form>""".format(collection_name)

    rename_collection = """
                    <form action="/rename?collection_name={0}" method="POST">
                        <input type='text' name='new name'>
                        <input type='submit' value='change name'>
                     </form>""".format(collection_name)

    return """  {0}
    <br>
    Image's in this collection<br>
    {1}
    <br>
    {2}
    <br>
    {3}
    <br>
    {4}
    <br>
    {5}
    <a href='/'> Back Home </a>

    """.format(collection_name, obj_link, add_range_collection, add_single_collection, sort_list, rename_collection)


@app.route('/obj', methods=["GET"])
def obj_view():
    obj_title = request.args.get("obj_title")
    collection_name = request.args.get('collection_name')
    collection = all_collections.get_collection(collection_name)
    obj = collection.get_obj(obj_title)
    return """
            <div>
                <h2>Title: {0}</h2>
                <p>Date: {1}</p>
                <p>Explanation: {2}</p>
                <div class="image-frame">
                    <img src="{3}" alt="{0}" style="max-width: 50%;">
                </div>
                <br>
                <a href='/collection?collection_name={4}'> Back to Collection </a>
            </div>
            """.format(obj.get_title(), obj.get_date(), obj.get_explanation(), obj.get_image(), collection_name)


@app.route('/createcollection', methods=["POST"])
def create_collection():
    collection_name = request.form.get('name')
    list_of_names = all_collections.get_list_of_names()
    collection_date = request.form.get('date')
    if collection_name in list_of_names:
        return """
            <html><body>
                        <p> <b> Unsuccessful! Collection name Invalid! </b></p>
                        <br></br>
                            <br><br><a href='/'> Back Home </a>
            </body></html>
        """
    elif check_date(collection_date) is False:
        return """
            <html><body>
                        <p> <b> Unsuccessful! Collection date Invalid! </b></p>
                        <br></br>
                            <br><br><a href='/'> Back Home </a>
            </body></html>
        """

    url = 'https://api.nasa.gov/planetary/apod?api_key=bYgUJCu6QjKcvu7WSAElha7ZFNzZq3Dibeq9vYMV&date={0}'.format(
        collection_date)
    response = requests.get(url)
    data = json.loads(response.text)
    try:
        apod = ImageJSON(data)
    except:
        return """
            <html><body>
                        <p> <b> Unsuccessful! No image for this date! </b></p>
                        <br></br>
                            <br><br><a href='/'> Back to home </a>
            </body></html>
        """
    collection = ImageCollection(collection_name)
    collection.add_obj(apod)
    all_collections.add(collection)
    show_obj = """
            <div>
                <h2>Title: {0}</h2>
                <p>Date: {1}</p>
                <p>Explanation: {2}</p>
                <div class="image-frame">
                    <img src="{3}" alt="{0}" style="max-width: 50%;">
                </div>
                <br>
            </div>
            """.format(apod.get_title(), apod.get_date(), apod.get_explanation(), apod.get_image())

    return """
        <html><body>
            {0}
            <br><br><a href='/'> Back Home </a>
        </body></html>
        """.format(show_obj)


@app.route('/add_range_collection', methods=["POST"])
def add_range_collection():
    start = request.form.get('start date')
    end = request.form.get('end date')
    collection_name = request.args.get('collection_name')

    if check_date(start) is False or check_date(end) is False or start > end:
        return """
            <html><body>
                        <p> <b> Unsuccessful! Collection date Invalid! </b></p>
                        <br></br>
                            <br><br><a href='/collection?collection_name={0}'> Back to collection </a>
            </body></html>
        """.format(collection_name)
    else:
        collection = all_collections.get_collection(collection_name)
        url = 'https://api.nasa.gov/planetary/apod?api_key=bYgUJCu6QjKcvu7WSAElha7ZFNzZq3Dibeq9vYMV&start_date={0}&end_date={1}'.format(
            start, end)
        response = requests.get(url)
        data = json.loads(response.text)
        for i in data:
            try:
                apod = ImageJSON(i)
            except:
                return """
                    <html><body>
                                <p> <b> Unsuccessful! No image for {0}! </b></p>
                                <br></br>
                                    <br><br><a href='/collection?collection_name={1}'> Back to collection </a>
                    </body></html>
                """.format(i['date'], collection_name)
            collection.add_obj(apod)
        return """
            <html><body>
                        <p> <b> Success! New images added to collection! </b></p>
                        <br></br>
                            <br><br><a href='/collection?collection_name={0}'> Back to collection </a>
            </body></html>
            """.format(collection_name)


@app.route('/add_single_collection', methods=["POST"])
def add_single_collection():
    date = request.form.get('date')
    collection_name = request.args.get('collection_name')
    collection = all_collections.get_collection(collection_name)

    if check_date(date) is False:
        return """
            <html><body>
                        <p> <b> Unsuccessful! Collection date Invalid! </b></p>
                        <br></br>
                            <br><br><a href='/collection?collection_name={0}'> Back to collection </a>
            </body></html>
        """.format(collection_name)
    else:
        url = 'https://api.nasa.gov/planetary/apod?api_key=bYgUJCu6QjKcvu7WSAElha7ZFNzZq3Dibeq9vYMV&date={0}'.format(
            date)
        response = requests.get(url)
        data = json.loads(response.text)
        try:
            apod = ImageJSON(data)
        except:
            return """
                <html><body>
                            <p> <b> Unsuccessful! No image for this date! </b></p>
                            <br></br>
                                <br><br><a href='/collection?collection_name={0}'> Back to collection </a>
                </body></html>
            """.format(collection_name)
        collection.add_obj(apod)
        return """
            <html><body>
                        <p> <b> Success! New images added to collection! </b></p>
                        <br></br>
                            <br><br><a href='/collection?collection_name={0}'> Back to collection </a>
            </body></html>
            """.format(collection_name)


@app.route('/sort', methods=['POST'])
def sort_obj_list():
    collection_name = request.args.get('collection_name')
    collection = all_collections.get_collection(collection_name)
    sort_by = request.form.get('sort_by')
    if sort_by == 'title':
        collection.sort_by_title()
        return """
                        <html><body>
                                    <p> <b> Success! Collection sorted by title! </b></p>
                                    <br></br>
                                        <br><br><a href='/collection?collection_name={0}'> Back to collection </a>
                        </body></html>
                        """.format(collection_name)
    elif sort_by == 'date':
        collection.sort_by_date()
        return """
                                <html><body>
                                            <p> <b> Success! Collection sorted by date! </b></p>
                                            <br></br>
                                                <br><br><a href='/collection?collection_name={0}'> Back to collection </a>
                                </body></html>
                                """.format(collection_name)


@app.route('/rename', methods=['POST'])
def rename():
    collection_name = request.args.get('collection_name')
    collection = all_collections.get_collection(collection_name)
    list_of_names = all_collections.get_list_of_names()
    new_name = request.form.get('new name')
    if new_name not in list_of_names:
        all_collections.rename_collection(collection, new_name)
        collection_name = collection.get_name()
        return """
                                    <html><body>
                                                <p> <b> Success! Collection renamed to {1}! </b></p>
                                                <br></br>
                                                    <br><br><a href='/collection?collection_name={0}'> Back to collection </a>
                                    </body></html>
                                    """.format(collection_name, new_name)
    else:
        return """
                                    <html><body>
                                                <p> <b> Unsuccessful! Collection name {1} taken! </b></p>
                                                <br></br>
                                                    <br><br><a href='/collection?collection_name={0}'> Back to collection </a>
                                    </body></html>
                                    """.format(collection_name, new_name)



if __name__ == "__main__":
    app.run(host="localhost", debug=True)

#, port=5001 in between "local host" and debug^^^

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://localhost:5000
Press CTRL+C to quit
 * Restarting with stat
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/gerardodelligatti/Documents/PythonProjects/NASA_API_PROGRAM/venv/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/gerardodelligatti/Documents/PythonProjects/NASA_API_PROGRAM/venv/lib/python3.11/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/Users/gerardodelligatti/Documents/PythonProjects/NASA_API_PROGRAM/venv/lib/python3.11/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/gerardodelligatti/Documents/PythonProjects/NASA_API_PROGRAM/venv/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 692, in initiali

SystemExit: 1

/Users/gerardodelligatti/Documents/PythonProjects/NASA_API_PROGRAM/venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
